# Analyse de la proportion de personnes utilisant au moins une fois le moteur de recherches

**Définition** : On souhaite avoir la proportion de personnes utilisant au moins une fois le moteur de recherches sur l'ensemble des visiteurs du site

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Récupération des données dans le dossier contenant les fichiers déjà DL

In [ ]:
from join_multiple_file import pd_read_pattern, PATH_CDTN_MONOLOG

logs_july = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-july/*")

In [ ]:
logs_july.shape

In [ ]:
logs_july.head()

In [ ]:
def filter_df_startwith_url(df: pd.DataFrame, url):
    return df[df['url'].str.match(f'^{url}', na=False)].copy()

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
def clean_dataset_from_useless_rows(df: pd.DataFrame):
    #df = df.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()
    return df[df['type'].isin(['visit_content', 'outil', 'themes', 'select_result', 'search', 'home',
                                    'pagecc_clickcontrib', 'modeles_courriers'])]

In [ ]:
logs_july = format_url_and_get_month_and_datetime_columns(logs_july)

In [ ]:
logs = logs_july.copy()

# Explorons les données 

D'abord on souhaite voir quelles sont les étapes composant une recherche

In [ ]:
list_user_having_used_search_engine = logs[logs['type']=='search'].idVisit.unique().tolist()

In [ ]:
# Affichons les parcours utilisateurs des 10 premiers visiteurs de la liste
for idVisit in list_user_having_used_search_engine[:10]:
    display(logs[logs['idVisit']==idVisit])

On déduit que les étapes qui composent une recherche sont : 
 - un log de type select_suggestion afin de sélectionner une des sélection pour compléter les mots entrés par l'utilisateur
 - un log de type search lorsque l'utilisateur tape dans la barre de recherche
 - un log de type select_result lors de la selection du résultat après avoir utilisé le moteur de recherche

### Préparation des données

In [ ]:
logs.head()

In [ ]:
logs_wihtout_duplicates = logs.drop_duplicates(subset=['idVisit', 'type']).copy()

### Nombre total de visits

In [ ]:
nb_visits = logs_wihtout_duplicates.idVisit.nunique()
nb_visits

### Nombre de sélection de suggestion lors de la recherche sur le moteur

In [ ]:
nb_visits_with_at_least_one_select_suggestion = logs_wihtout_duplicates.type.value_counts()['select_suggestion']
nb_visits_with_at_least_one_select_suggestion

### Nombre d'utilisateurs ayant utilisés le moteur de recherche

In [ ]:
nb_visits_with_at_least_one_search = logs_wihtout_duplicates.type.value_counts()['search']
nb_visits_with_at_least_one_search

### Nombre d'utilisateurs ayant sélectionné un résultat après avoir fait une recherche

In [ ]:
nb_visits_with_at_least_one_select_result = filter_df_startwith_url(
    logs_wihtout_duplicates, 'https://code.travail.gouv.fr/recherche').type.value_counts()['select_result']
nb_visits_with_at_least_one_select_result

### Présentation des résultats

In [ ]:
def round_2_dec(num, denum):
    return round(num / denum, 4)*100

In [ ]:
ratio = pd.DataFrame(
    columns = ['name', 'rate'],
    data = [
        ['Clic sur une suggestion', round_2_dec(nb_visits_with_at_least_one_select_suggestion, nb_visits)],
        ['Recherche', round_2_dec(nb_visits_with_at_least_one_search, nb_visits)],
        ['Clic sur un resultat', round_2_dec(nb_visits_with_at_least_one_select_result, nb_visits)],
    ]
)

### Présentation Graphique du résultat

In [ ]:
fig = px.histogram(
    ratio, x="name", y="rate",
    color='name', height=400,
    title="Taux d'utilisation du moteur de recherche (en %)"
)
fig.update_layout(
    yaxis_title="ratio",
    legend_title="Catégories",
)
fig.show()

## Nombre de recherches par utilisateurs ayant au moins fait une recherche
### Préparation des données

### Nombre total de visits

In [ ]:
nb_visits = logs[logs['type'].isin(['search', 'select_suggestion'])].idVisit.nunique()
nb_visits

### Nombre de sélection de suggestion lors de la recherche sur le moteur

In [ ]:
nb_visits_with_at_least_one_select_suggestion = logs.type.value_counts()['select_suggestion']
nb_visits_with_at_least_one_select_suggestion

### Nombre d'utilisateurs ayant utilisés le moteur de recherche

In [ ]:
nb_visits_with_at_least_one_search = logs.type.value_counts()['search']
nb_visits_with_at_least_one_search

### Nombre d'utilisateurs ayant sélectionné un résultat après avoir fait une recherche

In [ ]:
nb_visits_with_at_least_one_select_result = filter_df_startwith_url(
    logs, 'https://code.travail.gouv.fr/recherche').type.value_counts()['select_result']
nb_visits_with_at_least_one_select_result

### Présentation des résultats

In [ ]:
def round_2_dec(num, denum):
    return round(num / denum, 4)*100

In [ ]:
ratio = pd.DataFrame(
    columns = ['name', 'rate'],
    data = [
        ['Clic sur une suggestion', nb_visits_with_at_least_one_select_suggestion / nb_visits ],
        ['Recherche', nb_visits_with_at_least_one_search / nb_visits ],
        ['Clic sur un resultat', nb_visits_with_at_least_one_select_result / nb_visits ],
    ]
)

### Présentation Graphique du résultat

In [ ]:
fig = px.histogram(
    ratio, x="name", y="rate",
    color='name', height=400,
    title="Nombre moyen de recherches par utilisateur ayant fait au moins une recherche"
)
fig.update_layout(
    yaxis_title="Nombre de recherches",
    legend_title="Catégories",
)
fig.show()